In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric_temporal.nn.recurrent import EvolveGCNH, EvolveGCNO, GConvLSTM, GConvGRU
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

In [8]:
data_df = pd.read_csv("datasetsrip - synthetic-prcp-SM-flood-data-v1.csv.csv")
data_df.drop(['Unnamed: 0'], axis=1, inplace=True)
data_df

,Prcp1,SM1,Prcp2,SM2,Prcp3,SM3,flood1,flood2,flood3
0,1.539606,0.590441,2.511432,0.474052,4.585283,0.484656,0,0,0
1,2.274125,1.249220,7.461011,2.159259,17.690303,3.474562,1,1,1
2,1.000114,0.106579,1.842560,0.010000,3.395878,0.010000,0,0,0
3,1.360013,0.429366,2.085287,0.212607,3.610686,0.137334,0,0,0
4,1.158710,0.248820,1.282190,0.010000,1.592812,0.010000,0,0,0
...,...,...,...,...,...,...,...,...,...
360,1.357030,0.426691,2.020841,0.197245,3.431358,0.103488,0,0,0
361,1.154324,0.244887,3.097722,0.201381,6.761527,0.473908,0,0,1
362,3.315756,2.183443,2.927501,2.330118,3.422077,2.060617,1,1,0
363,1.779925,0.805979,2.532323,0.718383,4.327215,0.680561,0,0,0


EvolveGCNH

cpu
Epoch: 000, Loss: 100.00000
Epoch: 001, Loss: 0.00000
Epoch: 002, Loss: 0.00000
Epoch: 003, Loss: 0.00000
Epoch: 004, Loss: 0.00000
Epoch: 005, Loss: 0.00000
Epoch: 006, Loss: 0.00000
Epoch: 007, Loss: 0.00000
Epoch: 008, Loss: 0.00000
Epoch: 009, Loss: 0.00000
Epoch: 010, Loss: 0.00000
Epoch: 011, Loss: 0.00000
Epoch: 012, Loss: 0.00000
Epoch: 013, Loss: 0.00000
Epoch: 014, Loss: 0.00000
Epoch: 015, Loss: 0.00000
Epoch: 016, Loss: 0.00000
Epoch: 017, Loss: 0.00000
Epoch: 018, Loss: 0.00000
Epoch: 019, Loss: 0.00000
Test Accuracy: 0.5906


In [17]:
def test_1layer_EvolveGCNH(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, node_count, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNH(node_count, dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(node_count=3, dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(10):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNH(data_df)

cpu
Epoch: 000, Loss: 0.66538
Epoch: 001, Loss: 0.50111
Epoch: 002, Loss: 0.42760
Epoch: 003, Loss: 0.38522
Epoch: 004, Loss: 0.33729
Epoch: 005, Loss: 0.29453
Epoch: 006, Loss: 0.25325
Epoch: 007, Loss: 0.21780
Epoch: 008, Loss: 0.18972
Epoch: 009, Loss: 0.16714
Test Accuracy: 0.8949


In [18]:
def test_1layer_EvolveGCNH(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, node_count, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNH(node_count, dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(node_count=3, dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(20):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNH(data_df)

cpu
Epoch: 000, Loss: 0.28858
Epoch: 001, Loss: 0.12725
Epoch: 002, Loss: 0.09504
Epoch: 003, Loss: 0.06695
Epoch: 004, Loss: 0.05581
Epoch: 005, Loss: 0.04927
Epoch: 006, Loss: 0.04527
Epoch: 007, Loss: 0.04204
Epoch: 008, Loss: 0.04161
Epoch: 009, Loss: 0.04057
Epoch: 010, Loss: 0.03958
Epoch: 011, Loss: 0.03876
Epoch: 012, Loss: 0.03797
Epoch: 013, Loss: 0.03699
Epoch: 014, Loss: 0.03527
Epoch: 015, Loss: 0.03402
Epoch: 016, Loss: 0.03245
Epoch: 017, Loss: 0.03144
Epoch: 018, Loss: 0.03054
Epoch: 019, Loss: 0.02930
Test Accuracy: 0.8949


In [11]:
def test_1layer_EvolveGCNH(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, node_count, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNH(node_count, dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(node_count=3, dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(50):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNH(data_df)

cpu
Epoch: 000, Loss: 1.18914
Epoch: 001, Loss: 0.65057
Epoch: 002, Loss: 0.51335
Epoch: 003, Loss: 0.48344
Epoch: 004, Loss: 0.40577
Epoch: 005, Loss: 0.33092
Epoch: 006, Loss: 0.25335
Epoch: 007, Loss: 0.19519
Epoch: 008, Loss: 0.15482
Epoch: 009, Loss: 0.12758
Epoch: 010, Loss: 0.10769
Epoch: 011, Loss: 0.09208
Epoch: 012, Loss: 0.08023
Epoch: 013, Loss: 0.07123
Epoch: 014, Loss: 0.06433
Epoch: 015, Loss: 0.05808
Epoch: 016, Loss: 0.05233
Epoch: 017, Loss: 0.04697
Epoch: 018, Loss: 0.04205
Epoch: 019, Loss: 0.03767
Epoch: 020, Loss: 0.03376
Epoch: 021, Loss: 0.03029
Epoch: 022, Loss: 0.02725
Epoch: 023, Loss: 0.02087
Epoch: 024, Loss: 0.01914
Epoch: 025, Loss: 0.01764
Epoch: 026, Loss: 0.01641
Epoch: 027, Loss: 0.01543
Epoch: 028, Loss: 0.01462
Epoch: 029, Loss: 0.01393
Epoch: 030, Loss: 0.01333
Epoch: 031, Loss: 0.01287
Epoch: 032, Loss: 0.01247
Epoch: 033, Loss: 0.01216
Epoch: 034, Loss: 0.01193
Epoch: 035, Loss: 0.01180
Epoch: 036, Loss: 0.01171
Epoch: 037, Loss: 0.01170
Epoch: 0

In [12]:
def test_1layer_EvolveGCNH(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, node_count, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNH(node_count, dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(node_count=3, dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(100):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNH(data_df)

cpu
Epoch: 000, Loss: 0.18285
Epoch: 001, Loss: 0.08105
Epoch: 002, Loss: 0.04990
Epoch: 003, Loss: 0.03652
Epoch: 004, Loss: 0.03026
Epoch: 005, Loss: 0.02607
Epoch: 006, Loss: 0.02396
Epoch: 007, Loss: 0.02228
Epoch: 008, Loss: 0.02155
Epoch: 009, Loss: 0.02034
Epoch: 010, Loss: 0.02274
Epoch: 011, Loss: 0.02255
Epoch: 012, Loss: 0.02184
Epoch: 013, Loss: 0.02103
Epoch: 014, Loss: 0.02070
Epoch: 015, Loss: 0.02039
Epoch: 016, Loss: 0.02006
Epoch: 017, Loss: 0.02018
Epoch: 018, Loss: 0.01901
Epoch: 019, Loss: 0.02832
Epoch: 020, Loss: 0.02893
Epoch: 021, Loss: 0.03082
Epoch: 022, Loss: 0.03112
Epoch: 023, Loss: 0.03161
Epoch: 024, Loss: 0.03199
Epoch: 025, Loss: 0.03234
Epoch: 026, Loss: 0.00545
Epoch: 027, Loss: 0.00499
Epoch: 028, Loss: 0.00458
Epoch: 029, Loss: 0.00425
Epoch: 030, Loss: 0.00401
Epoch: 031, Loss: 0.00294
Epoch: 032, Loss: 0.00281
Epoch: 033, Loss: 0.00262
Epoch: 034, Loss: 0.00253
Epoch: 035, Loss: 0.00242
Epoch: 036, Loss: 0.00234
Epoch: 037, Loss: 0.00228
Epoch: 0

EvolveGCNO

In [23]:
def test_1layer_EvolveGCNO(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNO(dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(10):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNO(data_df)

cpu
Epoch: 000, Loss: 0.58584
Epoch: 001, Loss: 0.47106
Epoch: 002, Loss: 0.40163
Epoch: 003, Loss: 0.35636
Epoch: 004, Loss: 0.32642
Epoch: 005, Loss: 0.30763
Epoch: 006, Loss: 0.29698
Epoch: 007, Loss: 0.29234
Epoch: 008, Loss: 0.29177
Epoch: 009, Loss: 0.29356
Test Accuracy: 0.8370


In [14]:
def test_1layer_EvolveGCNO(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNO(dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(20):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNO(data_df)

cpu
Epoch: 000, Loss: 0.66866
Epoch: 001, Loss: 0.54168
Epoch: 002, Loss: 0.45534
Epoch: 003, Loss: 0.39684
Epoch: 004, Loss: 0.35636
Epoch: 005, Loss: 0.32810
Epoch: 006, Loss: 0.30625
Epoch: 007, Loss: 0.28767
Epoch: 008, Loss: 0.27266
Epoch: 009, Loss: 0.25989
Epoch: 010, Loss: 0.24859
Epoch: 011, Loss: 0.23860
Epoch: 012, Loss: 0.23025
Epoch: 013, Loss: 0.22472
Epoch: 014, Loss: 0.21988
Epoch: 015, Loss: 0.23432
Epoch: 016, Loss: 0.24615
Epoch: 017, Loss: 0.25420
Epoch: 018, Loss: 0.26015
Epoch: 019, Loss: 0.26412
Test Accuracy: 0.8587


In [29]:
def test_1layer_EvolveGCNO(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNO(dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(50):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNO(data_df)

cpu
Epoch: 000, Loss: 0.74748
Epoch: 001, Loss: 0.63592
Epoch: 002, Loss: 0.56896
Epoch: 003, Loss: 0.52576
Epoch: 004, Loss: 0.49545
Epoch: 005, Loss: 0.47256
Epoch: 006, Loss: 0.45433
Epoch: 007, Loss: 0.43930
Epoch: 008, Loss: 0.42657
Epoch: 009, Loss: 0.41559
Epoch: 010, Loss: 0.40599
Epoch: 011, Loss: 0.39751
Epoch: 012, Loss: 0.38995
Epoch: 013, Loss: 0.38317
Epoch: 014, Loss: 0.37707
Epoch: 015, Loss: 0.37155
Epoch: 016, Loss: 0.36656
Epoch: 017, Loss: 0.36202
Epoch: 018, Loss: 0.35790
Epoch: 019, Loss: 0.35415
Epoch: 020, Loss: 0.35073
Epoch: 021, Loss: 0.34761
Epoch: 022, Loss: 0.34477
Epoch: 023, Loss: 0.34217
Epoch: 024, Loss: 0.33979
Epoch: 025, Loss: 0.33762
Epoch: 026, Loss: 0.33564
Epoch: 027, Loss: 0.33383
Epoch: 028, Loss: 0.33216
Epoch: 029, Loss: 0.33064
Epoch: 030, Loss: 0.32925
Epoch: 031, Loss: 0.32797
Epoch: 032, Loss: 0.32679
Epoch: 033, Loss: 0.32572
Epoch: 034, Loss: 0.32473
Epoch: 035, Loss: 0.32383
Epoch: 036, Loss: 0.32300
Epoch: 037, Loss: 0.32223
Epoch: 0

In [33]:
def test_1layer_EvolveGCNO(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNO(dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(100):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.device.empty_cache()

test_1layer_EvolveGCNO(data_df)

cpu
Epoch: 000, Loss: 0.75795
Epoch: 001, Loss: 0.63142
Epoch: 002, Loss: 0.56094
Epoch: 003, Loss: 0.50083
Epoch: 004, Loss: 0.45305
Epoch: 005, Loss: 0.41518
Epoch: 006, Loss: 0.38493
Epoch: 007, Loss: 0.36048
Epoch: 008, Loss: 0.34046
Epoch: 009, Loss: 0.32390
Epoch: 010, Loss: 0.31007
Epoch: 011, Loss: 0.29842
Epoch: 012, Loss: 0.28853
Epoch: 013, Loss: 0.28008
Epoch: 014, Loss: 0.27281
Epoch: 015, Loss: 0.26652
Epoch: 016, Loss: 0.26102
Epoch: 017, Loss: 0.25618
Epoch: 018, Loss: 0.25187
Epoch: 019, Loss: 0.24801
Epoch: 020, Loss: 0.24454
Epoch: 021, Loss: 0.24143
Epoch: 022, Loss: 0.23866
Epoch: 023, Loss: 0.23619
Epoch: 024, Loss: 0.23398
Epoch: 025, Loss: 0.23200
Epoch: 026, Loss: 0.23023
Epoch: 027, Loss: 0.22863
Epoch: 028, Loss: 0.22719
Epoch: 029, Loss: 0.22589
Epoch: 030, Loss: 0.22471
Epoch: 031, Loss: 0.22363
Epoch: 032, Loss: 0.22265
Epoch: 033, Loss: 0.22175
Epoch: 034, Loss: 0.22092
Epoch: 035, Loss: 0.22017
Epoch: 036, Loss: 0.21947
Epoch: 037, Loss: 0.21883
Epoch: 0

AttributeError: type object 'torch.device' has no attribute 'empty_cache'